# Introduction
이번 장에서는 모델 성능의 더 나은 측정을 위한 교차 검증을 다룰 것이다. 

머신 러닝은 반복 작업이며 어떤 예측 변수를 사용할지, 어떤 유형의 모형을 사용할지, 어떤 인자를 모형에 제공할지 등을 선택해야 한다.   
지금까지는 검증(or holdout) 세트를 사용해 모델 품질을 측정해 데이터 중심적으로 선택했다. 

그러나 이 접근 방식에는 몇 가지 단점이 있다. 이를 보기위해 5000개의 행이 있는 데이터셋이 있다고 가정해보자.  
일반적으로 데이터의 약 20%를 유효성 검사 세트 또는 1000행으로 유지한다. 그러나 이는 모델 점수를 결정할 때 무작위성을 부여한다.   
즉, 다른 1000행에서 부정확하더라도 1000행의 한 집합에서는 모형이 잘 작동할 수 있다는 것이다. 

극단적으로 검증 세트에 데이터 행이 하나만 있는 경우를 상상해보자.  
대안 모델을 비교한다면, 단일 데이터 포인트에 대해 어떤 모델이 가장 좋은 예측을 하는지는 운에 달렸다고 할 수 있다. 

일반적으로 검증 세트가 클수록 모델 품질 특정에서 무작위성(일명 "노이즈")이 줄어 더 안정적이다.  
불행히도 훈련 데이터에서 행을 제거해야만 큰 검증 세트를 얻을 수 있으며, 훈련 데이터가 작으면 모델이 나빠진다. 

# What is cross-validation?
**Cross-validation(이하 "교차검증")**에서, 우리는 모델 품질의 여러 측정값을 얻기 위해 데이터의 다른 하위 집합에 대해 모델링 프로세스를 실행한다. 

예를 들어, 우리는 전체 데이터에서 각각 20%로 나눠지는 데이터를 5조각으로 나눌 수 있다. 
전체 데이터에서 각 20%인 5개 조각으로 나누는 것으로 시작할 수 있으며, 이를 데이터를 5개의 **'fold'**로 나눴다 한다. 
![nn](https://i.imgur.com/9k60cVA.png)

이후, 각각의 폴드에 대해 한개씩 실험을 진행한다.
1. **Experiment1(실험1)**
    - 전체 데이터에서 첫 번째 폴드를 검증(or holdout) 세트로, 그 외 전부는 훈련 데이터로 사용한다. 이는 20% 홀드아웃 세트를 기반으로 모델 품질을 측정한다. 
2. **Experiment2**
    - 두 번째 폴드를 제외한 모든 세이터를 사용해 모델을 훈련한다. 홀드아웃 세트는 모델 품질의 두 번째 추정치를 얻는 데 사용된다. 

3. 모든 폴드를 홀드아웃 세트로 한 번씩 사용해 이 프로세스를 반복한다. 이를 종합하면 데이터의 100%가 특정 시점에 홀드아웃으로 사용되며, 결국 데이터 세트의 모든 행을 기반으로 하는 모델 품질을 측정할 수 있다. 

# When should you use cross-validation?
교차 검증을 통해 모형 품질을 보다 정확하게 측정할 수 있으며, 이는 많은 모형들 사이에서 결정을 내리는 경우 특히 중요하다.  
그러나 여러 모형(폴드당 하나씩)을 추정하기에 실행하는데 많은 시간이 소요될 수 있다. 

*따라서, 이 점을 고려할 때 사용하는 경우는 무엇일까?*

1. 추가 계산 부담이 크지 않은 소규모 데이터 세트의 경우 
2. 대규모 데이터의 경우 단일 검증 세트로 충분하다. 
    - 코드 실행이 더 빠르며, 데이터를 충분히 보유하기에 일부 데이터를 보류용으로 재사용할 필요가 거의 없다.
    
데이터의 규모를 구분하는데 임계값은 없으나, 모델을 실행하는데 몇 분이 채 소요되지 않는다면 교차 검증으로 전환할 가치가 있다. 

또는 교차 검증을 통해 각 실험의 점수가 근접한지 확인할 수 있다.  
각 실험이 동일한 결과가 나오는 경우, 단일 검증 집합으로 충분할 것이다. 

# Example
이전 튜토리얼과 동일한 데이터로 작업할 것이다. 입력 데이터는 X로, 출력 데이터는 y로 로드한다.

In [5]:
import pandas as pd

# Read the data
data=pd.read_csv('./data/melb_data.csv')

# Select subset of predictors
cols_to_use=['Rooms','Distance','Landsize','BuildingArea','YearBuilt']
X=data[cols_to_use]

# Select target
y=data.Price

다음으로 'imputer'를 사용해 결측값을 채우고 'RandomForest' 모델을 사용해 예측을 수행하는 파이프라인을 정의한다.  
(파이프라인 없이 교차 검증을 수행하는 것을 가능하지만 꽤 어려우며, 파이프라인을 사용하면 코드가 간결해진다.)

In [7]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

pipes=Pipeline(steps=[
    ('preprocessor',SimpleImputer()),
    ('model',RandomForestRegressor(n_estimators=50, 
                                  random_state=0))
])

우리는 scikit-learn에서 **cross_val_score()** 함수를 사용해 교차 검증 점수를 얻을 것이다.  
(cv-fold 횟수 설정)

In [10]:
from sklearn.model_selection import cross_val_score

# Multiply by -1 since sklearn calculates *negative* MAE
# sklearn은 *부정*으로 계산하므로 결과에 -1을 곱해야 한다.
scores = -1 * cross_val_score(pipes, X, y, cv=5, scoring='neg_mean_absolute_error')

print(f'MAE scores :\n \033[94m{scores}')

MAE scores :
 [301628.7893587  303164.4782723  287298.331666   236061.84754543
 260383.45111427]


scoring parameter는 리포트할 모델 품질 측정값을 선택하며, 위 경우에서는 음의 <u>평균 절대 오차(MAE)</u>를 선택했다.  
(scikit-learn용 문서는 옵션 목록을 보여준다.). 

우리는 일반적으로 대안 모델을 비교하기 위해 모델 품질의 단일 측정값을 원하므로 실험 전반에 걸쳐 평균을 취한다. 

>**참고**<br>음의(negative) MAE를 지정한 것은 조금 놀라운 점이다. Scikit-learn은 모든 메트릭이 정의되어 높은 수가 더 좋다는 규칙을 가진다.  
음의 MAE는 다른 곳에서 들어 본 적은 없지만 여기서 음수를 사용하면 기존 규칙과 일치할 수 있다. 

In [12]:
print("Average MAE score (across experiments):")
print(f'\033[94m{scores.mean()}')

Average MAE score (across experiments):
277707.3795913405


# Conclusion
교차 검증을 사용하면 코드를 정리하는 이점이 더해져 모델 품질 측정이 용이해진다.  
즉, 더 이상의 별도의 훈련 및 검증 세트를 추적할 필요가 없어진다는 점에 유의한다.  
따라서, 특히 소규모 데이터셋의 경우, 이는 권할만한 사항이다. 